In [1]:
library(shazam)
library(tidyverse)

Loading required package: ggplot2

As of v1.0.0 the AIRR Rearrangement schema is now the default file format.
A description of the standard is available at https://docs.airr-community.org.
The legacy Change-O format is supported through arguments to each function
that allow the input column names to be explicitly defined.

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [6]:
# ----------------------- read data ---------------------------
sample <- read.csv('./data//GW/light_parse-select.tsv', header=T, sep='\t')  %>% as.tibble()


# --------------------- remove duplicate -----------------------
sample_index <- sample %>% group_by(cell_id) %>% summarise(umi_count=max(umi_count)) 
sample_filter <- sample[which(paste0(sample$cell_id,sample$umi_count) %in% paste0(sample_index$cell_id,sample_index$umi_count) ),]
# sample_filter %>% dim()

dup_index <- (table(sample_filter$cell_id) >1) %>% as.data.frame() %>% filter(.== TRUE) %>% rownames()
sample_single <-sample_filter[ !sample_filter$cell_id %in% dup_index, ]

In [7]:
dim(sample_single)
sample_single$locus %>% head()

[1] 6081   57

[1] "TRB" "TRB" "TRB" "TRB" "TRB" "TRB"

In [8]:
# -------------------- calculate distance -----------------------
dist_sc <- distToNearest(sample_single, cellIdColumn="cell_id", locusColumn="locus", VJthenLen=FALSE, onlyHeavy=FALSE, nproc =8)

output2 <- findThreshold(dist_sc$dist_nearest, method="gmm", model="gamma-gamma")

pdf(snakemake@output[[2]])
plot(output2, binwidth=0.02, title="GMM Method: gamma-gamma")
dev.off()


# -------------------- calculate distance -----------------------
distance <- output2@threshold
write(distance,snakemake@output[[1]])

ERROR: Error in is.factor(x): object 'snakemake' not found
